# Fine-Tune DistilBERT For Multi-Class Text Classification Using Tensorflow and Keras

In this project, we explore the capabilities of DistilBERT for multi-class text classification by comparing three approaches: no fine-tuning, standard fine-tuning, and LoRA fine-tuning.

Table of Contents:
    1. Data Pre-Processing
    2. Metrics We Will Measure
    3. Classification with No Fine-Tuning
    4. Classification with Standard Fine-Tuning
    5. Classification with LoRA Fine-Tuning
    

In [5]:
## Import required packages
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tf_keras
import tensorflow as tf
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from plotly.offline import iplot


/Users/bennettbishop/enterprise-email-routing/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Preprocess Data

### 1.1. Split into Train, Validation, Test using Stratified Sampling

In [6]:
# Import Data
root_path = 'data/full_dataset.csv'
df = pd.read_csv(root_path)
df.head()

# Encode the 'category' column into numerical labels
df['encoded_text'] = df['category'].astype('category').cat.codes

# Separate columns for splitting
data_texts = df['request'].to_list()  # 'request' is the text data
data_labels = df['encoded_text'].to_list()  # Encoded class labels
stratify_values = df['stratify_col'].to_list()  # Stratification column

# Split the data into Train/Validation sets with stratification
train_texts, val_texts, train_labels, val_labels, train_stratify, val_stratify = train_test_split(
    data_texts, data_labels, stratify_values, 
    test_size=0.2, stratify=stratify_values, random_state=0
)

# Split the Train set further into Train/Test with stratification
train_texts, test_texts, train_labels, test_labels = train_test_split(
    train_texts, train_labels, 
    test_size=0.1, stratify=train_stratify, random_state=0
)

### 1.2. View test/train/validation Splits

In [7]:
# Map numerical labels back to category names
label_mapping = dict(enumerate(df['category'].astype('category').cat.categories))
print("\nLabel Mapping (Encoded -> Category):")

for encoded, category in label_mapping.items():
    print(f"{encoded}: {category}")

# Output dataset information
print("\nFinal dataset information:")
print(f"Train set size: {len(train_texts)}")
print(f"Validation set size: {len(val_texts)}")
print(f"Test set size: {len(test_texts)}")

print(f"Example train_texts: {train_texts[:3]}") 
print(f"Example train_labels: {train_labels[:3]}")
print(f"Example val_texts: {val_texts[:3]}") 
print(f"Example val_labels: {val_labels[:3]}")
print(f"Example test_texts: {test_texts[:3]}") 
print(f"Example test_labels: {test_labels[:3]}")


Label Mapping (Encoded -> Category):
0: Facilities Management
1: Finance
2: HR
3: IT Support
4: Marketing

Final dataset information:
Train set size: 3596
Validation set size: 1000
Test set size: 400
Example train_texts: ['I’m gathering details about rewards for long-term employees and was hoping you could provide some insight. Let me know if you need further specifics from me.', 'Do you have the latest version of the diversity and inclusion policies handbook? I need it for a new hire orientation.', 'Could you share detailed insights on the performance metrics for keyword research for PPC campaigns? I’d like to use this data for our planning.']
Example train_labels: [2, 2, 4]
Example val_texts: ['Could you outline the steps to optimize our launching retargeting ads approach? Any case studies or examples would be helpful.', 'Need access to server maintenance.', 'Insights on customer retention strategies performance needed.']
Example val_labels: [4, 3, 4]
Example test_texts: ['I’m exper

### 1.3. Optional: Save test/train/val to CSV

In [224]:
# Create DataFrames for each split
train_df = pd.DataFrame({
    'request': train_texts,
    'label': train_labels
})

val_df = pd.DataFrame({
    'request': val_texts,
    'label': val_labels
})

test_df = pd.DataFrame({
    'request': test_texts,
    'label': test_labels
})

# Save DataFrames to CSV files
train_df.to_csv("data/train.csv", index=False)
val_df.to_csv("data/validation.csv", index=False)
test_df.to_csv("data/test.csv", index=False)


## 1.4. Tokenize Data For DistilBERT Model

In [16]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation = True, padding = True  )

val_encodings = tokenizer(val_texts, truncation = True, padding = True )

test_encodings = tokenizer(test_texts, truncation = True, padding = True )

In [22]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(32)


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(32)

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 37), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 37), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


## 2. Metrics to Evaluate

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define evaluation function
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, precision, recall, f1

## 2. Predict Categories Without Fine-Tuning

### 2.1 Setup Model

In [18]:
# Model setup
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)
opt = tf_keras.optimizers.legacy.Adam(learning_rate=5e-5)
loss = tf_keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Raw logits expected
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

## 2.2 Evaluate Performance of DistilBERT with no fine-tuning

In [ ]:


# Perform prediction on the test dataset
y_true = []
y_pred = []

# Iterate over test dataset to collect true labels and predictions
for batch in test_dataset:
    input_data, labels = batch
    y_true.extend(labels.numpy())  # Collect true labels
    logits = model.predict(input_data).logits  # Predict logits
    predictions = logits.argmax(axis=-1)  # Convert logits to predicted labels
    y_pred.extend(predictions)

# Evaluate the model
no_finetuning_accuracy, no_finetuning_precision, no_finetuning_recall, no_finetuning_f1 = evaluate_model(y_true, y_pred)

# Print the results
print(f"Accuracy: {no_finetuning_accuracy}")
print(f"Precision: {no_finetuning_precision}")
print(f"Recall: {no_finetuning_recall}")
print(f"F1 Score: {no_finetuning_f1}")

1/1 [==============================] - 0s 158ms/step
Accuracy: 0.16
Precision: 0.16400330757965964
Recall: 0.16
F1 Score: 0.1155358770903418


2024-12-30 12:34:38.873624: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/bennettbishop/enterprise-email-routing/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



## 3. Predict Categories With Standard Fine-Tuning

### Define and Train Model

In [29]:
# Model setup
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)
opt = tf_keras.optimizers.legacy.Adam(learning_rate=5e-5)
loss = tf_keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Raw logits expected
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])


# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3,
    verbose=1
) 

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
113/113 [==============================] - 99s 855ms/step - loss: 0.3511 - accuracy: 0.9274 - val_loss: 0.0446 - val_accuracy: 0.9890
Epoch 2/3
113/113 [==============================] - 96s 852ms/step - loss: 0.0208 - accuracy: 0.9972 - val_loss: 0.0045 - val_accuracy: 1.0000
Epoch 3/3
113/113 [==============================] - 106s 941ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000


1/1 [==============================] - 1s 770ms/step
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [35]:
train_set = set(train_texts)
val_set = set(val_texts)
test_set = set(test_texts)

print(f"Train-Test Overlap: {len(train_set & test_set)}")
print(f"Train-Val Overlap: {len(train_set & val_set)}")
print(f"Val-Test Overlap: {len(val_set & test_set)}")


Train-Test Overlap: 0
Train-Val Overlap: 0
Val-Test Overlap: 0


## Save Model

In [198]:
save_directory = "./saved_models" 

model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

('./saved_models/tokenizer_config.json',
 './saved_models/special_tokens_map.json',
 './saved_models/vocab.txt',
 './saved_models/added_tokens.json')